# Basic example

This example demonstrates some of the core functionality and export features provided by rabpro.

In [1]:
import pandas as pd
import geopandas as gpd

import rabpro

First, we need to specify a point within our test DEM region.

In [2]:
coords = (56.22659, -130.87974)

Now we can initialize the profiler. The rabpro profiler is the main entry point into the package - it provides wrapper funcitons for most of the rabpro's core functionality.

Note that we can optionally specify a drainage area (`da`) or set `force_merit` to `True`, to ensure that we use MERIT data rather than HydroBasins to perform basin delineation.

In [5]:
rpo = rabpro.profiler(coords, name='basic_test', rebuild_vrts=False)

Supplied path for building vrt: []


RuntimeError: The path you supplied appears empty.

rabpro can now compute the watershed for this point. This can take a few minutes to run.

In [ ]:
%%capture
rpo.delineate_basin()

The basin geometry is stored in a GeoPandas GeoDataFrame, and can be accessed through the `rpo` object.

In [ ]:
rpo.watershed.plot()

Next, we compute the river elevation profile.

In [ ]:
rpo.elev_profile()

The river elevation profile is stored in a GeoPandas GeoDataFrame called "flowline", and can be accessed through the `rpo` object.

In [ ]:
rpo.flowline.head()

The basins and river elevation profile can be exported to GeoJSON with the export command:

In [ ]:
%%capture
rpo.export("all")

Once the subbasins are delinated, rabpro can use Google Earth Engine (GEE) to compute statistics for each subbasin. Using Google Earth Engine reduces the need to store large datasets locally, and speeds up computation by using GEE's parallel distributed computing capabilities.

Note: In order to use rabpro for basin statistics, you'll need to sign up for a GEE account. See rabpro's documentation for more information.

In [ ]:
# Specify which statistics to calculate for the JRC/GSW1_3/GlobalSurfaceWater dataset's occurrence band
statlist = ['min', 'max', 'range', 'std', 'sum', 'pct50', 'pct3']
data = rabpro.basin_stats.Dataset("JRC/GSW1_3/GlobalSurfaceWater", "occurrence", stats=statlist)

d, t = rpo.basin_stats([data], folder="rabpro test")

The output data will be placed in the `rabpro test` folder in your Google Drive if it already exists. If not, GEE will create a new `rabpro test` folder at the root level of your Drive.

`basin_stats` returns a url to the resulting csv data which can be read directly with `pandas`:

In [ ]:
pd.read_csv(d[0])